<code> 
<strong><font size="+2">Saving MHW events from each ensemble member run</font></strong>
In this Jupyter notebook, we save the MHW events from each ensemble member run. This is a pipeline that allows us to create 'member0_events.nc' (and files like this) used in 3_clusteranalysispipeline.ipynb. In the notebook before this one, we ran Ocetrac on the 100 CESM-LE simulations, setting a radius size 3. 
</code>

<code> Import packages </code>

In [2]:
import xarray as xr; import numpy as np; import pandas as pd; 
import netCDF4 as nc; import datetime as dt
import scipy

import intake; import pprint
# Allow multiple lines per cell to be displayed without print (default is just last line)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Enable more explicit control of DataFrame display (e.g., to omit annoying line numbers)
from IPython.display import HTML

<code> Load marine heatwave event files and SST anomaly files </code>

In [4]:
%%time
 
# Open original collection description file #----------------------------------------------
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)

CPU times: user 24.9 s, sys: 1.44 s, total: 26.3 s
Wall time: 27.9 s


In [6]:
%%time

subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')
filenamechange = list(subset.df.member_id.unique())

CPU times: user 1.01 s, sys: 1.28 ms, total: 1.01 s
Wall time: 1.07 s


In [8]:
print(filenamechange)

['r1i1001p1f1', 'r2i1021p1f1', 'r3i1041p1f1', 'r4i1061p1f1', 'r5i1081p1f1', 'r6i1101p1f1', 'r7i1121p1f1', 'r8i1141p1f1', 'r9i1161p1f1', 'r10i1181p1f1', 'r1i1231p1f1', 'r2i1231p1f1', 'r3i1231p1f1', 'r4i1231p1f1', 'r5i1231p1f1', 'r6i1231p1f1', 'r7i1231p1f1', 'r8i1231p1f1', 'r9i1231p1f1', 'r10i1231p1f1', 'r1i1251p1f1', 'r2i1251p1f1', 'r3i1251p1f1', 'r4i1251p1f1', 'r5i1251p1f1', 'r6i1251p1f1', 'r7i1251p1f1', 'r8i1251p1f1', 'r9i1251p1f1', 'r10i1251p1f1', 'r1i1281p1f1', 'r2i1281p1f1', 'r3i1281p1f1', 'r4i1281p1f1', 'r5i1281p1f1', 'r6i1281p1f1', 'r7i1281p1f1', 'r8i1281p1f1', 'r9i1281p1f1', 'r10i1281p1f1', 'r1i1301p1f1', 'r2i1301p1f1', 'r3i1301p1f1', 'r4i1301p1f1', 'r5i1301p1f1', 'r6i1301p1f1', 'r7i1301p1f1', 'r8i1301p1f1', 'r9i1301p1f1', 'r10i1301p1f1', 'r1i1011p1f2', 'r2i1031p1f2', 'r3i1051p1f2', 'r4i1071p1f2', 'r5i1091p1f2', 'r6i1111p1f2', 'r7i1131p1f2', 'r8i1151p1f2', 'r9i1171p1f2', 'r10i1191p1f2', 'r11i1231p1f2', 'r12i1231p1f2', 'r13i1231p1f2', 'r14i1231p1f2', 'r15i1231p1f2', 'r16i1231p1f2

In [9]:
%%time

list_of_xarrays = []
list_of_xarrays_SSTA = []

for i in filenamechange:
    
    string_head = '/glade/work/cassiacai/' + str(i) + '_rad3_blobs.nc'    
    xarray_file = xr.open_dataset(str(string_head))
    list_of_xarrays.append(xarray_file)
    
    string_head_SSTA = '/glade/work/cassiacai/' + str(i) + '_detrended.nc'
    xarray_file_SSTA = xr.open_dataset(str(string_head_SSTA))
    list_of_xarrays_SSTA.append(xarray_file_SSTA)

CPU times: user 1.72 s, sys: 359 ms, total: 2.08 s
Wall time: 13.7 s


In [10]:
%%time

concated_xarray = xr.concat(list_of_xarrays, "new_dim")

CPU times: user 4.5 s, sys: 10.7 s, total: 15.2 s
Wall time: 59.7 s


In [11]:
%%time

concated_xarray_SSTA = xr.concat(list_of_xarrays_SSTA, "new_dim")

CPU times: user 4.28 s, sys: 10.8 s, total: 15.1 s
Wall time: 39 s


<code> Combine SSTA and MHW event files </code>

In [15]:
%%time

combined_xarray = xr.combine_by_coords([concated_xarray, concated_xarray_SSTA])
combined_xarray['SSTA'] = combined_xarray['__xarray_dataarray_variable__']
combined_xarray = combined_xarray.drop(['__xarray_dataarray_variable__'])

CPU times: user 3.4 ms, sys: 1.13 ms, total: 4.53 ms
Wall time: 30.7 ms


In [16]:
combined_xarray

<xarray.Dataset>
Dimensions:    (time: 481, lat: 192, lon: 288, new_dim: 100)
Coordinates:
  * time       (time) object 1975-01-01 00:00:00 ... 2015-01-01 00:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member_id  (new_dim) <U12 'r1i1001p1f1' 'r2i1021p1f1' ... 'r20i1301p1f2'
    quantile   float64 0.9
    month      (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12 1
Dimensions without coordinates: new_dim
Data variables:
    labels     (new_dim, time, lat, lon) float64 nan nan nan nan ... nan nan nan
    SSTA       (new_dim, time, lat, lon) float64 0.0 0.0 ... -0.007121 -0.007111

In [21]:
for i in range(100):
    member_ = combined_xarray.isel(new_dim = i)
    member_.to_netcdf('/glade/work/cassiacai/member'+str(i)+'_events.nc')